# Pore analysis testing

In [ ]:
# Is porespy installed correctly?
import porespy as ps
import matplotlib.pyplot as plt
im = ps.generators.blobs(shape=[500, 500], porosity=0.6, blobiness=2)
plt.imshow(im)
plt.show()

In [ ]:
# General notebook settings
import seaborn
# Set seaborn theme
seaborn.set_theme(context='notebook', style='ticks')

In [ ]:
plt.rcParams['figure.figsize']

In [ ]:
# Set figure defaults
plt.rc('image', cmap='gray', interpolation='nearest')  # Display all images in b&w and with 'nearest' interpolation
# scalefactor = 2
# plt.rcParams['figure.figsize'] = (16 // scalefactor, 9 // scalefactor)  # Size up figures a bit
# plt.rcParams['figure.dpi'] = 300

In [ ]:
# Set figure defaults
plt.rc('image', cmap='gray', interpolation='nearest')  # Display all images in b&w and with 'nearest' interpolation
scalefactor = 2
plt.rcParams['figure.figsize'] = (16 // scalefactor, 9 // scalefactor)  # Size up figures a bit
# plt.rcParams['figure.dpi'] = 300

In [ ]:
# Let's look at an image from Alina

In [ ]:
import platform
import os

In [ ]:
if 'Win' in platform.system():
    Root = 'F:/'
else:
    Root = '/media/habi/Fast_SSD'
Path = os.path.join(Root, 'Schmid BFH Methylcellulose')
print('Our base path is %s' % Path)

In [ ]:
import imageio

In [ ]:
import glob

In [ ]:
# Load middle image from one scan

In [ ]:
Folder = os.path.join(Path, 'Blobs', 'MC-Titanweiss', 'rec')

In [ ]:
ImageList = sorted(glob.glob(os.path.join(Folder, '*.png')))

In [ ]:
print('Found %s images in %s' % (len(ImageList), Folder))
print('Loading the middle one below')

In [ ]:
# Load test image
inputimage = imageio.imread(ImageList[len(ImageList)//2], pilmode='L')

In [ ]:
from matplotlib_scalebar.scalebar import ScaleBar
# Setup scale bar defaults
plt.rcParams['scalebar.location'] = 'lower right'
plt.rcParams['scalebar.frameon'] = False
plt.rcParams['scalebar.color'] = 'white'

In [ ]:
voxelsize = 1.5 # We usually read this from the log file, but here we do it 'quick and dirty'

In [ ]:
import skimage

In [ ]:
# Show test image
plt.imshow(skimage.exposure.equalize_adapthist(inputimage))
plt.title('%s, %s x %s px' % (os.path.basename(ImageList[len(ImageList)//2]),
                              inputimage.shape[0],
                              inputimage.shape[1]))
plt.gca().add_artist(ScaleBar(voxelsize,'um'))
plt.axis('off')
plt.show()

In [ ]:
# Use only central part of the image
crop = 1000
plt.subplot(121)
plt.imshow(inputimage)
plt.axhline(crop)
plt.axhline(inputimage.shape[0]-crop)
plt.axvline(crop)
plt.axvline(inputimage.shape[1]-crop)
plt.gca().add_artist(ScaleBar(voxelsize,'um'))
plt.axis('off')
plt.subplot(122)
croppedimage = inputimage[crop:-crop,crop:-crop]
plt.imshow(skimage.exposure.equalize_adapthist(croppedimage))
plt.title('Middle slice crop, %s x %s px' % (croppedimage.shape[0], croppedimage.shape[1]))
plt.gca().add_artist(ScaleBar(voxelsize,'um'))
plt.axis('off')
plt.show()

In [ ]:
# Calculate thresholds to separate into foam and background
threshold_iso = skimage.filters.threshold_isodata(croppedimage)
threshold_otsu = skimage.filters.threshold_otsu(croppedimage)

In [ ]:
# Display gray value histogram of image
histogram = plt.hist(croppedimage.ravel(),
                     bins='doane', # nice bin size selection
                     histtype='bar',
                     log=True,
                     label='Histogram',
                     color=seaborn.color_palette()[0])
plt.axvline(threshold_iso, label='Isodata-Threshold@%s' % threshold_iso, c=seaborn.color_palette()[1])
plt.axvline(threshold_otsu, label='Otsu-Threshold@%s' % threshold_otsu, c=seaborn.color_palette()[2])
plt.legend()
plt.title('Logarithmic grayvalue histogram with %s bins' % len(histogram[1]))
seaborn.despine()
plt.show()

In [ ]:
binarizedimage = croppedimage < threshold_iso  # porespy expects 'True' for features of interest, so we true stuff smaller than the threshold, e.g the air

In [ ]:
plt.subplot(121)
plt.imshow(croppedimage)
plt.title('Center of original slice, %s x %s px' % (croppedimage.shape[0], croppedimage.shape[1]))
plt.gca().add_artist(ScaleBar(voxelsize,'um'))
plt.axis('off')
plt.subplot(122)
plt.imshow(~binarizedimage) # Invert for displaying
plt.title('Binarized image, %s x %s px' % (binarizedimage.shape[0], binarizedimage.shape[1]))
plt.gca().add_artist(ScaleBar(voxelsize,'um'))
plt.axis('off')
plt.show()

In [ ]:
localthickness=ps.filters.local_thickness(binarizedimage, sizes=50)

In [ ]:
import numpy as np

In [ ]:
len(np.unique(localthickness))

In [ ]:
plt.subplot(131)
plt.imshow(localthickness, cmap='magma')
plt.gca().add_artist(ScaleBar(voxelsize,'um'))
plt.axis('off')
plt.subplot(132)
plt.imshow(localthickness/binarizedimage, cmap='magma')
plt.gca().add_artist(ScaleBar(voxelsize,'um'))
plt.axis('off')
plt.subplot(133)
plt.imshow(~binarizedimage)
plt.gca().add_artist(ScaleBar(voxelsize,'um'))
plt.axis('off')
plt.show()

In [ ]:
import scipy

In [ ]:
dt = scipy.ndimage.distance_transform_edt(binarizedimage)
distance = 100
peaks = skimage.feature.peak_local_max(dt, min_distance=distance)
# skeleton = skimage.morphology.skeletonize(skimage.morphology.dilation(binarizedimage))

In [ ]:
# plt.imshow(skeleton)

In [ ]:
plt.imshow(dt/binarizedimage, cmap='viridis')
plt.gca().add_artist(ScaleBar(voxelsize,'um'))
plt.axis('off')
for peak in peaks:
    plt.scatter(peak[1], peak[0], marker='x', c='white')
plt.title('Distance transformation with %s overlaid peaks > %s px' % (len(peaks), distance))
plt.show()

In [ ]:
pk = ps.filters.find_peaks(dt, r_max=50)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=[6, 3])

pk = ps.filters.find_peaks(dt=dt)
ax[0].imshow((dt/binarizedimage), cmap='viridis')
ax[0].axis(False)
plt.gca().add_artist(ScaleBar(voxelsize,'um'))

ax[1].imshow(dt/
             binarizedimage/
             ~skimage.morphology.dilation(skimage.morphology.dilation(skimage.morphology.dilation(pk))), cmap='viridis')
ax[1].axis(False);
plt.axis('off')
plt.show()

Pore size distribution as per https://nbviewer.org/github/PMEAL/porespy/blob/dev/examples/filters/tutorials/local_thickness.ipynb

In [ ]:
psd = ps.metrics.pore_size_distribution(localthickness,
                                        log=False,
                                        bins=25,
                                        voxel_size=1.5)  # give the voxel size in um, then we get back um :)
print(psd)

In [ ]:
plt.bar(x=psd.R, height=psd.pdf, width=psd.bin_widths)
plt.xlabel('Pore radius [um]')
plt.ylabel('Normalized volume fraction')

SNOW partitioning, based on https://nbviewer.org/github/PMEAL/porespy/blob/dev/examples/filters/tutorials/snow_partitioning.ipynb

In [ ]:
from skimage.morphology import binary_dilation
ps.visualization.set_mpl_style()
np.random.seed(1)

In [ ]:
snow_out = ps.filters.snow_partitioning(binarizedimage, r_max=40, sigma=0.4)
print(snow_out)

In [ ]:
dt_peak = snow_out.dt.copy()
peaks_dilated = binary_dilation(snow_out.peaks >> 0)
dt_peak[peaks_dilated > 0] = np.nan

In [ ]:
plt.subplot(221)
plt.imshow(~snow_out.im, cmap='gray')
plt.title('(inverted) Binary image')
plt.gca().add_artist(ScaleBar(voxelsize,'um'))
plt.axis('off')
plt.subplot(222)
plt.imshow(snow_out.dt)
plt.title('Distance transform')
plt.gca().add_artist(ScaleBar(voxelsize,'um'))
plt.axis('off')
plt.subplot(223)
plt.imshow(dt_peak)
plt.title("Distance transform peaks");
plt.gca().add_artist(ScaleBar(voxelsize,'um'))
plt.axis('off')
plt.subplot(224)
plt.imshow(ps.tools.randomize_colors(snow_out.regions)/binarizedimage)
plt.title("Segmentation")
plt.gca().add_artist(ScaleBar(voxelsize,'um'))
plt.axis('off')
plt.show()

Ab hier versuchen wir das in 3D

In [ ]:
from skimage.io import imread_collection

In [ ]:
seq = imread_collection('/media/habi/Fast_SSD/Schmid BFH Methylcellulose/Blobs/MC-Titanweiss/rec/TiO_rec00002[12]*.png')

In [ ]:
len(seq.files)

In [ ]:
from matplotlib.pyplot import subplots

In [ ]:
crop=100

In [ ]:
fig, ax = subplots(figsize=[5, 5])
ax.imshow(seq[0][1000+crop:-800-crop,600+crop:-1200-crop,0])

In [ ]:
im3d = np.zeros([*seq[0][1000+crop:-800-crop,600+crop:-1200-crop,0].shape, len(seq)])
for i, im in enumerate(seq):
    im3d[..., i] = im[1000+crop:-800-crop,600+crop:-1200-crop,0]

In [ ]:
im3d.shape

In [ ]:
ps.visualization.show_panels(im3d, axis=2)
plt.show()

In [ ]:
plt.imshow(ps.visualization.show_planes(im3d, spacing=150), cmap='magma')
plt.show()

In [ ]:
fig, ax = subplots(figsize=[5, 5])
ax.hist(im3d.flatten(), bins='doane', log=True);

In [ ]:
fig, ax = subplots(figsize=[5, 5])
ax.imshow(im3d[..., 0] < threshold_otsu);

In [ ]:
im = im3d < threshold_otsu
fig, ax = subplots(figsize=[5, 5])
ax.imshow(ps.visualization.sem(im, axis=2), cmap='grey');

In [ ]:
import numpy as np
import porespy as ps
import scipy.ndimage as spim
import matplotlib.pyplot as plt
ps.visualization.set_mpl_style()
np.random.seed(1)

In [ ]:
im = ps.generators.blobs(shape=[200, 200], porosity=0.6, blobiness=1)
fig, ax = plt.subplots()
ax.imshow(im, cmap=plt.cm.inferno);

In [ ]:
snow = ps.filters.snow_partitioning(im=im)
regions = snow.regions*snow.im

In [ ]:
fig, ax = plt.subplots()
ax.imshow(regions);

In [ ]:
props = ps.metrics.regionprops_3D(regions)

In [ ]:
r = props[0]
attrs = [a for a in r.__dir__() if not a.startswith('_')]
print(attrs)

In [ ]:
fig, ax = plt.subplots()
ax.imshow(r.image);

In [ ]:
fig, ax = plt.subplots()
ax.imshow(r.border + 0.5*r.inscribed_sphere);

In [ ]:
fig, ax = plt.subplots()
ax.imshow(r.image + 1.0*r.convex_image);

In [ ]:
print(f"Solidity: {r.solidity:.3f}")

In [ ]:
df = ps.metrics.props_to_DataFrame(props)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(8, 3))
ax[0].hist(df['volume'], edgecolor="k")
ax[1].hist(df['solidity'], edgecolor="k")
ax[2].hist(df['sphericity'], edgecolor="k");
ax[0].set_title("Volume")
ax[1].set_title("Solidity")
ax[2].set_title("Sphericity");

In [ ]:
df.iloc[0]

In [ ]:
# Create an image of maximally inscribed spheres
sph = ps.metrics.prop_to_image(regionprops=props, shape=im.shape, prop='inscribed_sphere')
fig, ax = plt.subplots()
ax.imshow(sph + 0.5*(~im) , cmap=plt.cm.inferno);

In [ ]:
# Create an image colorized by solidity
sph = ps.metrics.prop_to_image(regionprops=props, shape=im.shape, prop='solidity')
fig, ax = plt.subplots()
ax.imshow(sph + 0.5*(~im) , cmap='viridis');

In [ ]:
print('Done!')